# Import / Config

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

from IPython.display import Markdown, display

import edurel.utils.dbcon as dbcu
import edurel.utils.db as dbu
import edurel.utils.duckdb as ddbu
import edurel.utils.llm as llmu
import edurel.utils.llmchat as llmc
import edurel.widgets.mermaid_viz as mmw
import edurel.widgets.chatman as cmw
import edurel.utils.misc as mu

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases"


/home/iclassen/.py1/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


# Database

In [8]:
con = dbcu.playlist_de()
additional_fks = {}
# additional_fks = {"Orgeinheit": ["Mitarbeiter|Leitung|MID", "Orgeinheit|Obereinheit|OEID"]}

db = dbu.DbHandler(con,additional_fks=additional_fks)
db.schema_yaml_print(["nullable", "fkname"])

tables:
- tablename: Genre
  columns:
  - columnname: GID
    type: INTEGER
  - columnname: Bezeichnung
    type: VARCHAR
  primary_key:
  - GID
- tablename: Playlist
  columns:
  - columnname: PLID
    type: INTEGER
  - columnname: Name
    type: VARCHAR
  - columnname: LetzterAbruf
    type: DATE
  primary_key:
  - PLID
- tablename: Stueck
  columns:
  - columnname: SID
    type: INTEGER
  - columnname: Titel
    type: VARCHAR
  - columnname: DauerInSek
    type: INTEGER
  - columnname: PLID
    type: INTEGER
  - columnname: GID
    type: INTEGER
  primary_key:
  - SID
  foreign_keys:
  - sourcecolumns:
    - PLID
    targettable: Playlist
    targetcolumns:
    - PLID
  - sourcecolumns:
    - GID
    targettable: Genre
    targetcolumns:
    - GID

